<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/Randadomness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

stanford_rna_3d_folding_path = kagglehub.competition_download('stanford-rna-3d-folding')
metric_usalign_path = kagglehub.dataset_download('metric/usalign')
geraseva_protenix_checkpoints_path = kagglehub.dataset_download('geraseva/protenix-checkpoints')
biniroun_protenix_packages_path = kagglehub.dataset_download('biniroun/protenix-packages')
biniroun_linux_biowheels_310_path = kagglehub.dataset_download('biniroun/linux-biowheels-310')
biniroun_linux_mlwhl_311_path = kagglehub.dataset_download('biniroun/linux-mlwhl-311')
biniroun_protenix_main_path = kagglehub.dataset_download('biniroun/protenix-main')
biniroun_rdkit_cp310_path = kagglehub.dataset_download('biniroun/rdkit-cp310')
metric_ribonanza_tm_score_path = kagglehub.notebook_output_download('metric/ribonanza-tm-score')

print('Data source import complete.')


In [ ]:
!pip install --no-index --find-links=/kaggle/input/linux-biowheels-310 biopython numpy
!pip install --no-index --find-links=/kaggle/input/linux-mlwhl-311 ml-collections contextlib2
!pip install --no-index --find-links=/kaggle/input/linux-biowheels-310 biopython
!pip install --no-index --find-links=/kaggle/input/rdkit-cp310 rdkit
!pip install --no-index --find-links=/kaggle/input/linux-mlwhl-311 biopython ml-collections contextlib2 rdkit
!pip install --no-index --find-links=/kaggle/input/linux-mlwhl-311 biotite

In [ ]:
import sys
sys.path.append("/kaggle/input/protenix-main")  # 또는 업로드한 zip의 정확한 이름

# 이후 import 실행
from protenix.data.infer_data_pipeline import InferenceDataset

# 📌 기본 세팅
import os, sys, numpy as np, pandas as pd, torch, warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

# 📌 GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📌 Using device: {device}")

# 📌 USAlign 설정
os.system("cp /kaggle/input/usalign/USalign /kaggle/working/")
os.system("chmod u+x /kaggle/working/USalign")

# 📌 Protenix 코드 및 패키지 경로 등록
sys.path.append("/kaggle/input/protenix-main")
sys.path.append("/kaggle/input/protenix-packages")

# 📌 학습 데이터 병합 (v2 우선)
seq_v1 = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')
seq_v2 = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.v2.csv')
label_v1 = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv')
label_v2 = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.v2.csv')

seq_v1 = seq_v1[~seq_v1['target_id'].isin(seq_v2['target_id'])]
label_v1 = label_v1[~label_v1['ID'].str.extract(r"^(.+)_\d+$")[0].isin(label_v2['ID'].str.extract(r"^(.+)_\d+$")[0])]

merged_seq = pd.concat([seq_v2, seq_v1], ignore_index=True)
merged_label = pd.concat([label_v2, label_v1], ignore_index=True)

# 📌 입력 생성
inputs = []
for _, row in merged_seq.iterrows():
    tid = row["target_id"]
    seq = row["sequence"]
    lbl = merged_label[merged_label["ID"].str.startswith(tid + "_")].copy()
    lbl.sort_values("resid", inplace=True)
    if len(lbl) != len(seq):
        continue
    coords = lbl[["x_1", "y_1", "z_1"]].values.astype(np.float32)
    inputs.append({
        "name": tid,
        "sequences": [{
            "rnaSequence": {"sequence": seq, "count": 1},
            "coordinates": coords.tolist()
        }]
    })